In [1]:
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [2]:
data = pd.read_csv('data/hr_test.csv')

In [3]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [4]:
data = remove_random(data, .25)

In [5]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.57,4.0,270.0,4.0,0.0,1.0,0.0,technical,low
1,NaN,0.86,5.0,198.0,4.0,0.0,0.0,0.0,technical,low
2,NaN,0.97,5.0,197.0,5.0,1.0,0.0,0.0,technical,low
3,NaN,0.86,3.0,182.0,10.0,0.0,0.0,0.0,sales,low
4,NaN,0.82,4.0,276.0,2.0,0.0,0.0,0.0,IT,medium


In [6]:
imputer = impyte.Imputer()

In [7]:
print imputer.is_nan(["", 'None', 'NaN'])
print imputer.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


[True, False, False]
[True, True, True]
